<a href="https://colab.research.google.com/github/bismilyah/123/blob/main/Homework_%22practice_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендуется выполнять в гугл колаб
В этом задании вам нужно будет подготовить загрузку данных в раг, подготовить классификатор запросов, и настроить rule-based логику агента с использованием LLM.

In [ ]:
!pip install -U pip
!pip install -U "transformers>=4.50.0" accelerate sentencepiece huggingface_hub
!pip -q install langchain torch faiss-cpu langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 186.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 192.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 M

In [2]:
from typing import List
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch, textwrap, json


In [3]:
# --------------------
# 1. Документы (только Япония)
# --------------------
raw_json=[
  {
    "id": 1,
    "title": "JR Pass в Японии",
    "text": "JR Pass — это проездной билет для неограниченных поездок на поездах сети Japan Railways в течение выбранного периода."
  },
  {
    "id": 2,
    "title": "Как получить визу в Японию гражданину Казахстана",
    "text": "Для туристической визы в Японию гражданам Казахстана требуется подтверждение маршрута, финансов и обратные билеты."
  }
]
data=raw_json
print("Документов о Японии:",len(data))


Документов о Японии: 2


In [4]:
# --------------------
# 2. Индекс LangChain
# --------------------
EMB_MODEL="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
emb=HuggingFaceEmbeddings(model_name=EMB_MODEL)

def chunk(text:str,size:int=200,overlap:int=50)->List[str]:
    words=text.split()
    return [" ".join(words[i:i+size]) for i in range(0,len(words),size-overlap)]

docs=[]
for d in data:
    for ch in chunk(d["text"]):
        docs.append(Document(page_content=ch,metadata=d))

vectorstore = FAISS.from_documents(docs, emb) #TODO вызовите FAISS и запишите туда документы
print("Чанков в индексе:",vectorstore.index.ntotal)


<ipython-input-4-0b8bba053db9>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb=HuggingFaceEmbeddings(model_name=EMB_MODEL)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Чанков в индексе: 2


In [5]:
# --------------------
# 3. Классификация japan/general
# --------------------
CLS="joeddav/xlm-roberta-large-xnli"
from transformers import pipeline
zero_clf = pipeline("zero-shot-classification", model=CLS)  #загрузите пайплайн с моделью для классификации

LABELS=["japan","general"]
TEMPLATE="Этот вопрос относится к Японии: {}"

def classify(question:str)->str:
    return zero_clf(question,LABELS,hypothesis_template=TEMPLATE)["labels"][0]


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
from huggingface_hub import login


# ▸ На странице модели https://huggingface.co/google/gemma-3-4b-it
#   нажмите "Agree and Access".
# ▸ В «Settings → Access Tokens» создайте токен и вставьте ниже.
access_token = "sunrise"  # ваш токен
login()



In [7]:
# -------------------------------
# 4. LLM  ▸ Gemma-3-4B-it
# -------------------------------
MODEL_ID = "google/gemma-3-4b-it"     # (или GPTQ-квант если нужно)

from transformers import (
    AutoTokenizer,                 # для чата можно использовать AutoTokenizer
    Gemma3ForConditionalGeneration # НОВЫЙ класс вместо AutoModelForCausalLM
)
import torch
from transformers import AutoTokenizer

tokenizer_llm = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_auth_token=True, # добавил данную строку, чтобы использовал токен
    use_fast=False # TODO загрузите токенайзер. Передайте use_fast=False
)

model_llm = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",            # автоматический мэппинг на GPU / CPU
    torch_dtype=torch.bfloat16    # на T4 можно заменить на torch.float16
).eval()

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [8]:
def generate_llm_answer(prompt: str, max_tokens: int = 256) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": prompt}
    ]

    input_ids = tokenizer_llm.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model_llm.device)

    input_len = input_ids.shape[-1]

    # 2. Генерируем
    with torch.inference_mode():
        """
       out_ids = model_llm.generate(input_ids, temperature=0.01)[0]
       #TODO вызовите генерацию модели с температурой 0.01
        """
        out_ids = model_llm.generate()[0]

    # 3. Декодируем только сгенерированную часть
    answer = tokenizer_llm.decode(
        out_ids[input_len:], skip_special_tokens=True
    )
    return answer.strip()

In [9]:
def retrieve(question:str,k:int=3)->str:
    docs=vectorstore.similarity_search(question,k=k)
    return "\n".join(d.page_content for d in docs)


In [10]:
# --------------------
# 5. Агент
# --------------------
def agent(question: str) -> str:
    label = classify(question)
    """TODO напишите логику формирования промта. Если классификатор отдал Японию, то идем в раг. В остальном случае нет.
    Если идем в раг, то сначала вытаскиваем контекст. Вытащенный контекст подставляем в промт. Промт для рага должен состоять из Контекст, Вопрос, Ответ
    Если не идем в раг, то должны просто подготовить промт с Вопрос, Ответ
    """
    if label == "japan":
        context = retrieve(question)
        prompt = f"Контекст: {context}\nВопрос: {question}\nОтвет:"
    else:
        prompt = f"Вопрос: {question}\nОтвет:"
    return generate_llm_answer(prompt)
    print(agent("Какая самая высокая гора в Японии?"))

In [11]:
# --------------------
# 6. Демо
# --------------------
demo_japan=["Что такое Japan Rail Pass и как им пользоваться?",
            "Нужна ли виза россиянам для поездки в Японию?"]
demo_general=["Что такое трансформер","Сколько дней в феврале?"]

for q in demo_japan+demo_general:
    print("="*80)
    print("Вопрос:",q)
    print("Ответ:",agent(q).replace('<end_of_turn>', ''),"\n")


Вопрос: Что такое Japan Rail Pass и как им пользоваться?
Ответ:  

Вопрос: Нужна ли виза россиянам для поездки в Японию?
Ответ:  

Вопрос: Что такое трансформер
Ответ:  

Вопрос: Сколько дней в феврале?
Ответ:  

